<a href="https://colab.research.google.com/github/orcunsaatcii/benzer_kitap_tahmini/blob/main/kitaplar_tfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simplemma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 MB 7.9 MB/s eta 0:00:00


In [ ]:
from simplemma import text_lemmatizer
import pandas as pd
import numpy as np
import ast
import re

In [ ]:

def clean_text(text):
    text = text.replace("Â", "a")
    text = text.replace("â", "a")
    text = text.replace("î", "i")
    text = text.replace("Î", "ı")
    text = text.replace("İ", "i")
    text = text.replace("I", "ı")
    text = text.replace(u"\u00A0", " ")
    text = text.replace("|", " ")

    text = re.sub(r"@[A-Za-z0-9]+", " ", text)
    text = re.sub(r"(.)\1+", r"\1\1", text)
    text = re.sub(r"https?:\/\/\S+", " ", text)
    text = re.sub(r"http?:\/\/\S+", " ", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"#(\w+)", " ", text)
    text = re.sub(r"^\x00-\x7F]+", " ", text)
    text = re.sub(r"[^A-Za-zâîığüşöçİĞÜŞÖÇ]+", " ", text)
    text = re.sub(r"((https://[^\s]+))", " ", text)
    text = " ".join(text.lower().strip().split())

    return " ".join(text_lemmatizer(text, lang="tr"))

In [ ]:

with open("books.txt", 'r', encoding="utf-8") as dosya:

    kitap_adlari_listesi = dosya.readlines()

In [ ]:



with open("plots.txt", 'r', encoding="utf-8") as dosya:
    # Dosyadaki her satiri okuma ve listeye ekleme
    kitap_ozetleri_listesi = dosya.readlines()

In [ ]:
data = {
    "kitap_adi": kitap_adlari_listesi,
    "ozet": kitap_ozetleri_listesi
}

indexes = [range(1,len(kitap_adlari_listesi)+1)]

df = pd.DataFrame(data=data)

In [ ]:
df.head()

,kitap_adi,ozet
0,1. Kürk Mantolu Madonna\n,"""Her gün, daima öğleden sonra oraya gidiyor, k..."
1,2. Şeker Portakalı\n,"Yazarlıkta karar kılıncaya kadar, boks antrenö..."
2,4. Hayvan Çiftliği\n,"İngiliz yazar George Orwell (1903-1950), ülkem..."
3,5. Satranç\n,"Satranç sonsuz eski, ama aynı zamanda sonrasız..."
4,6. Simyacı\n,"Simyacı, dünyaca ünlü Brezilyalı yazar Paulo C..."


In [ ]:
def clean_row(row):
    ozet = clean_text(row["ozet"])
    return ozet

In [ ]:
clean_row(df.iloc[1])

'yazar karar kıl kadar boks antrenörlük ressam ve heykeltıraş model yap muz plantasyon hamallık gece kulüp garsonluk kadar çeşit iş çalış jose mauro de vasconcelos başyapıt şeker portakal gün biri acı keşfet küçük bir çocuk öykü dür çok yoksul bir aile oğul ol dünya gel dokuz yaş yüz öğren bir gün yüz şampiyon ol hayal kur vasconcelos çocukluk derin izle taşı şeker portakal yaşam bekle değişim karşı büyük sarsıntı yaşa küçük zezenin baş geçen anlat vasconcelos tam on iki gün yaz bu roman yirmi yıl fazla bir zaman yürek taşı söyle aydın emeçin güzel türkçe dil armağan et şeker portakal başkahraman zezenin büyü yaşa serüven yazar güneş uyan ve delifişek ad roman izle'

In [ ]:
df["clean"] = df.apply(lambda row: clean_row(row), axis=1)
df.head()

,kitap_adi,ozet,clean
0,1. Kürk Mantolu Madonna\n,"""Her gün, daima öğleden sonra oraya gidiyor, k...",her gün daima öğle sonra ora git koridor resim...
1,2. Şeker Portakalı\n,"Yazarlıkta karar kılıncaya kadar, boks antrenö...",yazar karar kıl kadar boks antrenörlük ressam ...
2,4. Hayvan Çiftliği\n,"İngiliz yazar George Orwell (1903-1950), ülkem...",ingiliz yazar george orwell ülke daha çok bind...
3,5. Satranç\n,"Satranç sonsuz eski, ama aynı zamanda sonrasız...",satranç son eski ama aynı zaman sonra yeni kur...
4,6. Simyacı\n,"Simyacı, dünyaca ünlü Brezilyalı yazar Paulo C...",simyacı dünya ünlü brezilya yazar paulo coelho...


In [ ]:
kitaplar = df["clean"].values


In [ ]:
word2idx = {}
i = 0

for kitap in kitaplar:
    for sozcuk in kitap.split():
        if sozcuk not in word2idx:
            word2idx[sozcuk] = i
            i += 1

In [ ]:
kitap_sayisi = len(kitaplar)
sozluk_buyuklugu = len(word2idx)

print(kitap_sayisi, sozluk_buyuklugu)

1375 12947


In [ ]:
tf = np.zeros((kitap_sayisi, sozluk_buyuklugu))

for i, kitap in enumerate(kitaplar):
    sozcukler = kitap.split()
    for sozcuk in sozcukler:
        tf[i, word2idx[sozcuk]] += 1 # sözcük sayma

    tf[i] = tf[i] / len(sozcukler) # normalizasyon

print(tf[0])

[0.01123596 0.01123596 0.01123596 ... 0.         0.         0.        ]


In [ ]:
doc_freq = np.sum(tf > 0, axis=0)
print(doc_freq)

idf = np.log(kitap_sayisi / doc_freq)
print(idf)

[517 411  18 ...   1   1   1]
[0.97816614 1.2076158  4.33583725 ... 7.22620901 7.22620901 7.22620901]


In [ ]:
tf_idf = tf * idf

In [ ]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
def kitap2str(row_index):
    kitap = df.iloc[row_index]
    adi = kitap["kitap_adi"]
    ozet = re.sub(r"\n+", "\n", kitap["ozet"]).strip()
    if len(ozet) > 200:
        ozet = ozet[:200] + "..."

    return f"Kitap adı: {adi} \nÖzeti: {ozet}"


print(kitap2str(3))

Kitap adı: 5. Satranç
 
Özeti: Satranç sonsuz eski, ama aynı zamanda sonrasız yenidir; kuruluşu mekanik, ancak sadece hayalgücü ile etkilidir; geometrik açıdan sabit bir alanla sınırlı olmakla birlikte kombinasyonlarında sınırsızdı...


In [ ]:
def benzerlerini_bul(kitap_adi, benzer_kitap_sayisi):
    kitap_indexi = df[df["kitap_adi"] == kitap_adi].index[0]
    print("Aranan Kitap:")
    print(kitap2str(kitap_indexi))
    print()
    print("* " * 30)
    print()

    benzerlikler = []
    sorgu_kitap = tf_idf[kitap_indexi]
    for vec in tf_idf:
        benzerlik = cos_sim(sorgu_kitap, vec)
        benzerlikler.append(benzerlik)

    en_benzer_kitap_indexleri = (- np.array(benzerlikler)
                                ).argsort()[:benzer_kitap_sayisi + 1]
    print("Benzer Kitaplar:\n")
    for i, kitap_idx in enumerate(en_benzer_kitap_indexleri[1:]):
        print(str(i+1) + ") " + kitap2str(kitap_idx))
        print()
        print("- " * 20)
        print()

In [ ]:
benzerlerini_bul("103. Nutuk\n", 3)

Aranan Kitap:
Kitap adı: 103. Nutuk
 
Özeti: Muhterem Efendiler, sizi, günlerce işgal eden, uzun ve teferruatlı beyanatım, en nihayet, mazi olmuş bir devrin hikâyesidir. Bunda, milletim için ve müstakbel evlâtlarımız için dikkat ve teyakkuzu dav...

* * * * * * * * * * * * * * * * * * * * * * * * * * * * * * 

Benzer Kitaplar:

1) Kitap adı: 714. Safahat
 
Özeti: Halkının içinden yükselmiş, ama halkın içinde kalmış bir şair; kendi milleti kadar, Doğu'yu ve Batı'yı dili, edebiyatı ve müziği ile bilen bir aydın; Veterinerlik Fakültesi'nin ilk öğrencisi ve birinc...

- - - - - - - - - - - - - - - - - - - - 

2) Kitap adı: 832. Türkçülüğün Esasları
 
Özeti: Türk düşünce, kültür ve siyaset tarihinin önemli simalarından olan Ziya Gökalp, Türkçülüğün Esasları adlı eseriyle “Türk milletindenim” demenin ne demek olduğunu, Türk milletinin kim olduğunu, nereden...

- - - - - - - - - - - - - - - - - - - - 

3) Kitap adı: 293. Gazi Mustafa Kemal Atatürk
 
Özeti: “Tarihin akışını değiştiren, ona müh